# Import Modules & Setup

In [ ]:
!pip install openai pandas tqdm python-dotenv

In [ ]:
import openai
import csv
import pandas as pd
import time
import random
from tqdm.notebook import tqdm
import sys
import os
from dotenv import load_dotenv

In [ ]:
output_csv_file = '..\\dataset\\hn_gh_ai_story_sentiment.csv'

### Initialize OpenAI API

In [ ]:
# Set your OPENAI_API_KEY as an environment variable and then load
load_dotenv()

# Initialize the OpenAI client with your API key
api_key_openai = os.getenv("OPENAI_API_KEY")
client_openai = openai.OpenAI(api_key=api_key_openai)

### Prompt template

In [ ]:
prompt_template = '''
Please perform Sentiment Classification task. 
Given the story headline from Hacker News, assign a sentiment label expressed by the author towards "Artificial Intelligence (AI)" from ['negative': -1, 'neutral': 0, 'positive': 1]. 
Return label only and the reasoning the following format.

Headline: ML surveillance camera fails to detect criminals
Label:-1
Reason:The headline highlights an ML-powered tool's failure to perform its job well, potentially causing issues or public concern.
Headline: Show HN: "JAVIN" an AI for solving math problem
Label:0
Reason:The headline simply announces the "JAVIN" project as a math problem solver without clear positive implications.
Headline: AI solves International Math Olympiad problems at silver medal level
Label:1
Reason:The headline shows AI strong capability in solving difficult math problems achieving silver medal level.
Headline: Why wasting time developing AI? I wish the whole world stop using AI for good
Label:-1
Reason:The headline indicates that developing AI is a waste of time and wishes that AI usage would stop for good.
Headline: AI don't have right to ownership in the US
Label:0
Reason:The headline states facts about ownership or copyright law in the US. It does not clearly support or downgrade AI usage or development.
Headline: Show HN: "TEXIE," an AI self-driving car that makes your life easier.
Label:1
Reason:The headline advertises an AI self-driving project by claiming that it makes your life easier.

Headline: {headline_input}
Label:
Reason:
'''

### Functions

In [ ]:
def get_sentiment_and_reason_openai(title):
    prompt = prompt_template.format(headline_input=title)

    for _ in range(3):
        try:
            response = client_openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are an expert at analyzing text."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=150,
                temperature=0
            )
            result = response.choices[0].message.content.strip()
            #print(result)

            sentiment_line = next((line for line in result.split('\n') if "label:" in line.lower()), None)
            reason_line = next((line for line in result.split('\n') if "reason:" in line.lower()), None)

            if sentiment_line:
                try:
                    # Extract sentiment value and clean it
                    sentiment_str = sentiment_line.split(":")[1].strip()
                    # Handle negative numbers properly
                    if sentiment_str in ['-1', '0', '1']:
                        sentiment = int(sentiment_str)
                    else:
                        print(f"Invalid sentiment for title: {title}")
                        print(f"Raw sentiment line: {sentiment_line}")
                        print(f"Raw reason line: {reason_line}")
                        raise ValueError(f"Invalid sentiment value: {sentiment_line}")

                    reason = reason_line.split(":", 1)[1].strip()
                    return sentiment, reason

                except ValueError as e:
                    print(f"Warning: Could not parse sentiment: {e}")
                    return -999, "Error parsing sentiment"

            else:
                raise ValueError("Invalid response format")

        except Exception as e:
            print(f"API call failed: {e}. Retrying in 1-2 minutes...")
            time.sleep(random.randint(60, 120))

    return -999, "Failed after 3 retries"

In [ ]:
"""Function to process story titles and save results"""

def process_titles(df, output_csv_file, save_interval=5):
    # Resume from existing file, if available
    try:
        existing_df = pd.read_csv(output_csv_file)
        df.update(existing_df)  # Merge saved progress
        print("Resuming from the last saved point...")
    except FileNotFoundError:
        print("No existing output file found. Starting fresh...")

    pbar = tqdm(total=len(df), desc="Processing story titles")

    for i, row in df.iterrows():
        # Skip rows already processed
        if pd.notna(row.get('story_sentiment')) and pd.notna(row.get('story_sentiment_reason')):
            pbar.update(1)
            continue

        # Get sentiment and reason for the story title
        sentiment, reason = get_sentiment_and_reason_openai(row['title'])
        df.at[i, 'story_sentiment'] = sentiment
        df.at[i, 'story_sentiment_reason'] = reason
        print(f"sentiment: {sentiment}")
        print(f"reason: {reason}")
        pbar.update(1)

        # Save progress at intervals
        if (i + 1) % save_interval == 0:
            df.to_csv(output_csv_file, index=False)
            print(f"\nSaved progress at row {i + 1}")
        time.sleep(0.5)

    pbar.close()

    # Final save
    df.to_csv(output_csv_file, index=False)
    print(f"Final result saved to {output_csv_file}")

# Sentiment Analysis

In [ ]:
df = pd.read_csv(output_csv_file)

# Add columns if missing
if 'story_sentiment' not in df.columns:
    df['story_sentiment'] = None
if 'story_sentiment_reason' not in df.columns:
    df['story_sentiment_reason'] = None

# Process titles
process_titles(df, output_csv_file)

print(f'Sentiment analysis completed and saved to {output_csv_file}')